In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.api as sm
from statsmodels.api import OLS
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from scipy import stats
import matplotlib.pyplot as plt
import nltk.sentiment.vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from bs4 import BeautifulSoup 
import os
import time
import requests
import re
#nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment import SentimentAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
pd.set_option('display.max_columns', None) 

# Sentiment Word Analysis 8-Ks

In [6]:
mdict = pd.read_csv('C:\\Users\\whill\\OneDrive\\Desktop\\MGT of Fin Institutions\\Hills_A12\\LoughranMcDonald_MasterDictionary_2020.csv')
mdict["Positive"] = mdict["Positive"].apply(pd.to_numeric, errors = 'coerce')
mdict["Negative"] = mdict["Negative"].apply(pd.to_numeric, errors = 'coerce')
negatives = mdict[mdict['Negative'] != 0]['Word']
positives = mdict[mdict['Positive'] != 0]['Word']

,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Complexity,Syllables,Source
0,AARDVARK,1,312,1.422050e-08,1.335201e-08,3.700747e-06,96,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.367356e-10,8.882163e-12,9.362849e-09,1,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,4.102067e-10,1.200533e-10,5.359747e-08,7,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,15,6.836779e-10,4.080549e-10,1.406914e-07,14,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,8009,3.650384e-07,3.798698e-07,3.523914e-05,1058,0,0,0,0,0,0,0,0,3,12of12inf


In [4]:
years = [1995 + x for x in range(0,26)]

In [22]:
regex = r"(?<!\S)[A-Za-z]+(?!\S)|(?<!\S)[A-Za-z]+"
tonelist = []
for y in years:
    orgpath = 'C:\\Users\\whill\\OneDrive\\Desktop\\MGT of Fin Institutions\\Hills_A12\\Datafiles'
    cfiles = os.listdir('\\'.join([orgpath,str(y),'8_K_files']))
    for file in cfiles: 
        newfilepath = '\\'.join([orgpath,str(y),'8_K_files',file])
        filename = str(file).split('_')[0]
        with open(newfilepath) as f:
            txt = f.read()
            txt = txt.upper()
            txt = re.findall(regex, txt)
            pos = 0 
            neg = 0
            for words in negatives:
                neg += txt.count(words)
            for words in positives: 
                pos += txt.count(words)
            tonescore = (pos - neg)/len(txt)
            tonelist.append((filename, tonescore))
    print(f'{y} has been read')
tdf = pd.DataFrame(tonelist)
tdf = tdf.set_axis(['CIK', 'Tone Score'], axis = 1,inplace = False)
tdf.head()

1995 has been read
1996 has been read
1997 has been read
1998 has been read
1999 has been read
2000 has been read
2001 has been read
2002 has been read
2003 has been read
2004 has been read
2005 has been read
2006 has been read
2007 has been read
2008 has been read
2009 has been read
2010 has been read
2011 has been read
2012 has been read
2013 has been read
2014 has been read
2015 has been read
2016 has been read
2017 has been read
2018 has been read
2019 has been read
2020 has been read


,CIK,Tone Score
0,101771,-0.018425
1,104030,-0.012346
2,19617,-0.006452
3,201499,-0.005801
4,201513,0.004380


In [23]:
tdf.describe()

,Tone Score
count,5181.000000
mean,-0.004299
std,0.008990
min,-0.096852
25%,-0.007557
50%,-0.001547
75%,0.000335
max,0.035980


In [24]:
tdf.to_csv('C:\\Users\\whill\\OneDrive\\Desktop\\MGT of Fin Institutions\\Hills_A12\\tonescores.csv')

# Sentiment Sentence Analysis 8-Ks

In [33]:
sentencelist = []
for y in years:
    orgpath = 'C:\\Users\\whill\\OneDrive\\Desktop\\MGT of Fin Institutions\\Hills_A12\\Datafiles'
    cfiles = os.listdir('\\'.join([orgpath,str(y),'8_K_files']))
    for file in cfiles: 
        newfilepath = '\\'.join([orgpath,str(y),'8_K_files',file])
        filename = str(file).split('_')[0]
        with open(newfilepath) as f:
            txt = f.read()
            txt = txt.upper()
            sentences = sent_tokenize(txt)
            SIA = SentimentIntensityAnalyzer()
            ct = 0 
            wt = 0
            for sentence in sentences:
                ct += 1
                siascore = SIA.polarity_scores(sentence)['compound']
                wt += siascore
            overallscore = wt/ct
            sentencelist.append((filename, overallscore))
    print(f'{y} has been read')
sdf = pd.DataFrame(sentencelist)
sdf = sdf.set_axis(['CIK', 'Sentence Score'], axis = 1,inplace = False)
sdf.head()

1995 has been read
1996 has been read
1997 has been read
1998 has been read
1999 has been read
2000 has been read
2001 has been read
2002 has been read
2003 has been read
2004 has been read
2005 has been read
2006 has been read
2007 has been read
2008 has been read
2009 has been read
2010 has been read
2011 has been read
2012 has been read
2013 has been read
2014 has been read
2015 has been read
2016 has been read
2017 has been read
2018 has been read
2019 has been read
2020 has been read


,CIK,Sentence Score
0,101771,0.071157
1,104030,0.283350
2,19617,0.632813
3,201499,0.453689
4,201513,0.306962


In [34]:
sdf.to_csv('C:\\Users\\whill\\OneDrive\\Desktop\\MGT of Fin Institutions\\Hills_A12\\sentencescores.csv')

# Sentiment Analysis Extra Credit Bert Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained("ipuneetrathore/bert-base-cased-finetuned-finBERT")
model = AutoModelForSequenceClassification.from_pretrained("ipuneetrathore/bert-base-cased-finetuned-finBERT")

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

In [7]:
berttonelist = []
for y in years:
    orgpath = 'C:\\Users\\whill\\OneDrive\\Desktop\\MGT of Fin Institutions\\Hills_A12\\Datafiles'
    cfiles = os.listdir('\\'.join([orgpath,str(y),'8_K_files']))
    for file in cfiles: 
        newfilepath = '\\'.join([orgpath,str(y),'8_K_files',file])
        filename = str(file).split('_')[0]
        with open(newfilepath) as f:
            txt = f.read()
            txt = txt.upper()
            sentences = sent_tokenize(txt)
            ct = 0 
            wt = 0
            for sentence in sentences:
                ct += 1
                mlength = 100
                encodednew = tokenizer.encode_plus(sentence, add_special_tokens = True, max_length = mlength,
                                                    pad_to_max_length = True, return_attention_mask = True, return_tensors = 'pt')
                input_ids = (encodednew['input_ids'])
                attentionmask = (encodednew['attention_mask'])
                input_ids = torch.cat([input_ids], dim=0)
                attentionmask = torch.cat([attentionmask], dim=0)
                new_output = model(input_ids, token_type_ids=None,
                                      attention_mask=attentionmask)
                logits = new_output[0]
                predicted = logits.detach().numpy()
                flat_pred = np.concatenate(predicted, axis=0)
                new_pred = np.argmax(flat_pred).flatten()
                ptscore = new_pred[0]
                wt += ptscore
            overallscore = wt/ct
            berttonelist.append((filename, overallscore))
    print(f'{y} has been read')
bdf = pd.DataFrame(berttonelist)
bdf = bdf.set_axis(['CIK', 'Bert Score'], axis = 1,inplace = False)
bdf.head()

KeyboardInterrupt: 

In [ ]:
bdf.to_csv('C:\\Users\\whill\\OneDrive\\Desktop\\MGT of Fin Institutions\\Hills_A12\\bertscores.csv')